In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle, yaml
from pathlib import Path

In [ ]:
k = 1.2
# plt.rcParams["figure.figsize"] = [2*k * 6.4, k * 4.8]
plt.rcParams["figure.figsize"] = [1*k * 6.4, k * 4.8]

In [ ]:
from phystem.systems.ring.quantities.datas import DeltaData

def get_delta_configs(configs):
    return configs["run_cfg"].func_cfg["delta"]

def plot_graph(id, data: DeltaData, mode: str, debug_path: str=None):
    xlims = get_delta_configs(data.configs)["xlims"]
    space_cfg = data.configs["space_cfg"]

    if mode == "init":
        selected_mask = np.isin(data.init_uids[id], data.init_selected_uids[id])

        cms = data.init_cms[id]
        x, y = data.init_cms[id].T
        xs, ys = data.init_cms[id][selected_mask].T
        uids = data.init_uids[id]
        time =data.init_times[id]
    elif mode == "final":
        selected_mask = np.isin(data.final_uids[id], data.init_selected_uids[id])

        cms = data.final_cms[id]
        x, y = data.final_cms[id].T
        xs, ys = data.final_cms[id][selected_mask].T
        uids = data.final_uids[id]
        time =data.final_times[id]
    
    plt.title(f"{id} | t={time:.3f}")
    
    h = space_cfg.height
    plt.plot(2*[xlims[0]], [-h/2, h/2], color="black")
    plt.plot(2*[xlims[1]], [-h/2, h/2], color="black")

    plt.scatter(x, y)
    plt.scatter(xs, ys, color="red")

    if debug_path:
        from phystem.systems.ring.quantities.calculators import DeltaCalculator
        from matplotlib.collections import LineCollection

        debug = DeltaCalculator.load_debug_data(debug_path, id)
        
        links_lines = LineCollection(cms[debug.links], color="black")
        plt.gca().add_collection(links_lines)

    for i, uid in enumerate(uids):
        plt.annotate(str(uid), (x[i], y[i]))

    if mode == "final":
        wait_dist = get_delta_configs(data.configs)["wait_dist"]
        space_cfg = data.configs["space_cfg"]
        x_end = data.init_cms[id][:, 0].mean()  + wait_dist
        h = space_cfg.height
        plt.plot(2*[x_end], [-h/2, h/2], color="green")


def final_graph(id, data: DeltaData):
    wait_dist = get_delta_configs(data.configs)["wait_dist"]
    for uid, cms in data.final_cms[id].items():
        plt.figure()
        plt.title(f"{uid} | t = {data.final_times[id][uid]:.3f}")
        
        init_idx = np.where(data.init_uids[id] == uid)[0][0]
        x_end = data.init_cms[id][init_idx][0] + wait_dist
        space_cfg = data.configs["space_cfg"]
        h = space_cfg.height
        plt.plot(2*[x_end], [-h/2, h/2], color="black")

        x, y = cms.T
        plt.scatter(x, y)
        
        selected_idx = np.where(data.final_uids[id][uid] == uid)[0][0]
        xs, ys = cms[selected_idx]
        plt.scatter(xs, ys, color="red")

        for i, uid in enumerate(data.final_uids[id][uid]):
            plt.annotate(str(uid), (x[i], y[i]))
    
def check_selected_uids(data: DeltaData):
    print(data.num_points)
    for id in range(data.num_points):
        expected = sorted(data.init_uids[id])
        
        f_cms = sorted(data.final_cms[id].keys())
        if f_cms != expected:
            print("========")
            print(f"Error: final cm | id={id}")
            print("init:", expected)
            print("final:", f_cms)

        f_uids = sorted(data.final_uids[id].keys())
        if f_uids != expected:
            print("========")
            print(f"Error: final uids | id={id}")
            print("init:", expected)
            print("final:", f_uids)

def check_uids(data: DeltaData):
    for i in range(data.num_points_completed):
        if (data.final_uids[i] != data.init_uids[i]).any():
            print(f"Erro: id={i}")

In [ ]:
data = DeltaData("../p0/datas/init_test/delta")

In [ ]:
check_uids(data)

In [ ]:
# id = 10

# init_graph(id, data)
# final_graph(id, data)
# plot_graph(1, data, "init", debug_path="../p0/results/init_test/delta")


debug_path = "../p0/results/init_test/delta"
# debug_path = None
for i in range(min(data.num_points_completed, 4)):
    plt.figure()
    plt.subplot(121)
    plt.gca().set_aspect("equal")
    plot_graph(i, data, "init", debug_path=debug_path)
    plt.subplot(122)
    plt.gca().set_aspect("equal")
    plot_graph(i, data, "final", debug_path=debug_path)


In [ ]:
from phystem.systems.ring.quantities.calculators import DeltaCalculator

calc = DeltaCalculator("datas/delta", edge_k=1.3, root_path="results/delta")
calc.crunch_numbers()

In [ ]:
plt.plot(calc.times, calc.deltas, ".-")
plt.show()